In [1]:
!pip install pydantic langchain-teddynote langchain_community langchain_huggingface langchain_openai

from google.colab import output

output.clear()

In [2]:
import os

from google.colab import userdata

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [3]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



# 데이터 불러오기

In [4]:
bl_data_sampled=pd.read_csv('bl_sample_data_us.csv')

display(bl_data_sampled)

,HIST_SN,ISO_WD2_NAT_CD,HSCD,BASE_DE,IMP_CO_NAME,SUPLY_CO_NAME,EXP_NAT_CD,NAT_NAME,PRDT_DC_VAL,SM_IMP_AMT
0,11448275,US,210690,20201123,HEALTHYPAC 740 SPEEDWELL,MEBOL GF S.A.C.,LA,"LA, LAO",2700 BOXES FROZEN STRAWBERRY (FRESA CONGELADA)...,0.0
1,1799681,US,210690,20200225,JMD LLC,HALDIRAM SNACKS PVT. LTD.,IN,"IN, INDIA",FOOD PREPARATION NAMKEENS/PUFF S IEC NO. : 059...,0.0
2,3359945,US,210690,20200424,WATERMARK FOODS,SURYAMITRA EXIM (P) LIMITED,IN,"IN, INDIA",CARGO STOWED IN A REFRIGER ATED CONTAINER AT M...,0.0
3,8175385,US,210690,20200731,THAI INDOCHINE TRADE INC,THAI AGRI FOODS PUBLIC,TH,"TH, THAILAND","SHIPPER S LOAD, COUNT, SEAL AND WEIGHT 2X20 CY...",84000.0
4,967847,US,210690,20200115,HANDS INVESTMENT LLC,"""ROYAL TEA TAIWAN CO., LTD.""",TW,"""TW, TAIWAN""",416PKGS S.T.C. 414CTNS + 2WDCS FOOD & UTENSI L...,0.0
...,...,...,...,...,...,...,...,...,...,...
2995,10694110,US,210690,20201028,KENDALL FROZEN FRUITS,AGROINDUSTRIAL VALLE FRIO S.A.,BR,"BR, BRAZIL",560 CASES FROZEN IQF BERRY MEDLEY NO STRAWBERR...,0.0
2996,5849236,US,210690,20200729,JMD LLC,HALDIRAM SNACKS PVT. LTD,IN,"IN, INDIA",FOOD PREPARATION NAMKEENS/SWEE TS IEC NO. : 05...,0.0
2997,14108508,US,210690,20201018,HOUSE OF SPICES INDIA INC,NaN,IN,"IN, INDIA",HS CODE 21069099 AUT HENTIC INDIAN SNACK S 3 S...,0.0
2998,2866252,US,210690,20200419,JF HILLEBRAND JAPAN KK,JF HILLEBRAND ITALIA SPA,IT,"IT, ITALY",900 CASE(S) 000000004404011038 - BACO 12X75 JP...,791748.0


In [5]:
bk_data_sampled=pd.read_excel('BK_210690.xlsx')

display(bk_data_sampled.head())

,pid,product_name,company_name,product_description,url
0,3727772,Young Long Vegan Berry VitaminC,YoungLong Corp,“You need vitamin C!”\n • Vitamin C is involv...,https://buykorea.org/ec/prd/selectGoodsDetail....
1,3726032,"Pharmsville Co.,Ltd",Devil Diet All in One Shot,(2Tablet+20ml) *7ea =1 box\n1 day 1ea\n✅ FEATU...,https://buykorea.org/ec/prd/selectGoodsDetail....
2,3729265,"Kids Jelly (Calcium, Colostrum, Red Ginseng)","N FOOD CO.,LTD",3 kinds or Kids Jelly Sticks\ncalcium / colost...,https://buykorea.org/ec/prd/selectGoodsDetail....
3,3728686,Respiratory Probiotics (GC Wellbeing L. planta...,Soulcare Nutrition CO.LTD,• Product name: Lactobacillus plantarum\n\n• B...,https://buykorea.org/ec/prd/selectGoodsDetail....
4,3710739,Funeat Real Kombucha Peach Flavor Detox Drink ...,corporation realwhite,https://funeat.co.kr/shopinfo/company.html\n• ...,NaN


# Langsmith , LLM API KEY 설정

In [6]:
from langchain_teddynote import logging

# logging.langsmith("KOTRA_BL_data_matching")

In [7]:
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


os.environ["OPENAI_API_KEY"] = userdata.get('KOTRA2_openai_api_key')


# API 사용 rate 제한 설정


In [8]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter_openai = InMemoryRateLimiter(
    requests_per_second=8,  # maximum query per day - 10000 (tier 1)
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

<ipython-input-8-d541afd0af05>:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter_openai = InMemoryRateLimiter(


# 사용 모델 설정

In [9]:
# chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini", rate_limiter=rate_limiter_openai)
chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini")

# OpenAI API 이용 결과 (model : chatgpt 4o-mini)

# buyKOREA 설명 처리 프롬프트문

---------------------

+ 업무 배경 설명 (제품명, 상품 설명에 대한 데이터임을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 제품명과 상품 정보에 기반한 제품 설명 생성
+ 지시 내용별 예시 제공 (one-shot)

In [10]:
keyword_num=5

class DescriptionSummary(BaseModel):
    type : str = Field(description="product type")
    keyword : list[str] = Field(description=f"product keywords, which focus on product's main features")
    phrase : list[str] = Field(description=f"product phrases, which focus on product's main features")

parser= PydanticOutputParser(pydantic_object=DescriptionSummary)

In [11]:
summary_prompt_buykorea = PromptTemplate.from_template(
    """You are given data about product's description(PRDT_DESC) and product name(PRDT_NAME).
    All products fall under HS Code 842952, which is a subcategory of 8429.
    The description for 8429 is as follows:
    84.29 Self-propelled bulldozers, angledozers, graders, levellers, scrapers, mechanical shovels, excavators, shovel loaders, tamping machines and road rollers.

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DESC using both PRDT_NAME and PRDT_DESC.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Example :
    PRDT_DESC : 'FIORESE Pure Hydra Water Cream contains Centella Asiatica and Hyaluronic Acid, effectively delivering hydration and providing instant soothing benefits.
    With Niacinamide and Adenosine, it is a dual-functional cosmetic for brightening and wrinkle care. This cream helps create healthy, smooth, and radiant skin.
    The lightweight, hydrating formula absorbs quickly, delivering moisture deep into the skin with a fresh, non-sticky finish. Suitable for all skin types, especially sensitive, dull, and uneven skin.

    For Sensitive Skin: Gentle, irritation-free formula, suitable even for sensitive skin.

    FIORESE Brand: Fiorese is dedicated to clean beauty, using ingredients sourced from nature. 100% cruelty-free.'

    Output Example : 'The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients.'

    PRODUCT DESCRIPTION(PRDT_DESC) : {prdt_desc}
    PRODUCT NAME(PRDT_NAME) : {prdt_name}
    """
)


summary_chain_buykorea = summary_prompt_buykorea | chat_openai

In [12]:
buykorea_preprocessed_dict_openai={}

for index, row in bk_data_sampled.iterrows():
  response_buykorea = summary_chain_buykorea.invoke({"prdt_desc": row['product_description'], "prdt_name": row['product_name']})
  print(response_buykorea.content)
  buykorea_preprocessed_dict_openai[row['product_name']]=[response_buykorea.content]

The Young Long Vegan Berry Vitamin C is a dietary supplement designed to provide essential vitamin C, which plays a crucial role in collagen synthesis, the primary protein in connective tissue. This product acts as an antioxidant, helping to protect cells from oxidative stress. It is particularly beneficial for individuals who may not get enough vitamin C from their diet, including those who consume fewer fruits and vegetables, modern individuals with nutrient deficiencies, pregnant women needing support for iron absorption, and anyone seeking to enhance their overall vitality. Additionally, this product is vegan-certified, making it a conscientious choice for those who prioritize plant-based options.
The Pharmsville Co., Ltd product is a dietary supplement that comes in a box containing 7 packs, each with 2 tablets and 20ml of liquid. It is designed to boost metabolism and assist with weight management, featuring a blend of vitamins and minerals, including Botion (5000%), Vitamin B2 (

In [13]:
extract_prompt_buykorea = PromptTemplate.from_template(
    """Extract product type and keywords, phrases from input, based on following rules:
    - Strictly exclude any information about capacity or volume
    - Generate a list of keywords related to product's main feature, but strictly limit the number to {{keyword_num}} or less. Do not exceed this limit under any circumstances.
    - Generate a list of phrases related to product's main feature.

    INPUT : {input}
    FORMAT :{{ "type": "<product type>", "keyword": [<keyword1>. <keyword2>, ...], "phrase" : [<phrase1>. <phrase2>, ...]}}
    """
)

extract_prompt_buykorea= extract_prompt_buykorea.partial(keyword_num=keyword_num)

extract_chain_buykorea = extract_prompt_buykorea | chat_openai | parser

In [14]:
for key, value in buykorea_preprocessed_dict_openai.items():
  response_buykorea = extract_chain_buykorea.invoke({"input": value})
  print(response_buykorea)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.type)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.keyword)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.phrase)

type='dietary supplement' keyword=['vitamin C', 'antioxidant', 'vegan'] phrase=['essential vitamin C', 'collagen synthesis', 'protect cells from oxidative stress', 'beneficial for nutrient deficiencies', 'supports iron absorption', 'enhance overall vitality', 'vegan-certified']
type='dietary supplement' keyword=['metabolism', 'weight management', 'natural', 'vitamins', 'minerals'] phrase=['boost metabolism', 'assist with weight management', 'blend of vitamins and minerals', 'KFDA-approved ingredient', 'reduce body fat', 'free from artificial flavoring']
type='Kids Jelly' keyword=['nutritious', 'tasty', 'healthy'] phrase=['designed specifically for children', 'supports balanced growth and nutrition', 'easy to eat', 'sweet fruit flavor', 'boosts immunity']
type='Respiratory Probiotics' keyword=['Lactobacillus plantarum', 'respiratory function', 'allergic inflammation'] phrase=['support respiratory function', 'improve allergic inflammation', 'relief from bronchial phlegm', 'promoting resp

In [15]:
buykorea_preprocessed_dict_openai

{'Young Long Vegan Berry VitaminC': ['The Young Long Vegan Berry Vitamin C is a dietary supplement designed to provide essential vitamin C, which plays a crucial role in collagen synthesis, the primary protein in connective tissue. This product acts as an antioxidant, helping to protect cells from oxidative stress. It is particularly beneficial for individuals who may not get enough vitamin C from their diet, including those who consume fewer fruits and vegetables, modern individuals with nutrient deficiencies, pregnant women needing support for iron absorption, and anyone seeking to enhance their overall vitality. Additionally, this product is vegan-certified, making it a conscientious choice for those who prioritize plant-based options.',
  'dietary supplement',
  ['vitamin C', 'antioxidant', 'vegan'],
  ['essential vitamin C',
   'collagen synthesis',
   'protect cells from oxidative stress',
   'beneficial for nutrient deficiencies',
   'supports iron absorption',
   'enhance overa

In [16]:
buykorea_preprocessed_openai=pd.DataFrame.from_dict(buykorea_preprocessed_dict_openai, orient='index', columns=['description_summary', 'type', 'keyword', 'phrase'])
buykorea_preprocessed_openai.reset_index(inplace=True)
buykorea_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
buykorea_preprocessed_openai.to_csv('buykorea_preprocessed_openai.csv', index=False)

display(buykorea_preprocessed_openai.head())

,product_name,description_summary,type,keyword,phrase
0,Young Long Vegan Berry VitaminC,The Young Long Vegan Berry Vitamin C is a diet...,dietary supplement,"[vitamin C, antioxidant, vegan]","[essential vitamin C, collagen synthesis, prot..."
1,"Pharmsville Co.,Ltd","The Pharmsville Co., Ltd product is a dietary ...",dietary supplement,"[metabolism, weight management, natural, vitam...","[boost metabolism, assist with weight manageme..."
2,"Kids Jelly (Calcium, Colostrum, Red Ginseng)","The Kids Jelly (Calcium, Colostrum, Red Ginsen...",Kids Jelly,"[nutritious, tasty, healthy]","[designed specifically for children, supports ..."
3,Respiratory Probiotics (GC Wellbeing L. planta...,"The GC Wellbeing Respiratory Probiotics, conta...",Respiratory Probiotics,"[Lactobacillus plantarum, respiratory function...","[support respiratory function, improve allergi..."
4,Funeat Real Kombucha Peach Flavor Detox Drink ...,The Funeat Real Kombucha Peach Flavor Detox Dr...,Detox Drink,"[kombucha, probiotic, detoxification]","[promote health and wellness, supports digesti..."


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ 업무 배경 설명 (수입자, 공급자, 상품 설명이 존재함을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 공급자와 상품 이름 정보에 기반한 제품 설명 생성
+ 인터넷 검색으로 추가 정보 검색 지시
+ 의미가 확실치 않은 경우 생성 제한
+ 지시 내용별 예시 제공 (one-shot)

In [17]:
generate_prompt_bldata = PromptTemplate.from_template(
    """You are given data where IMP_CO_NAME imports products (PRDT_DC_VAL) from SUPLY_CO_NAME.
    All products fall under HS Code 842952, which is a subcategory of 8429.
    The description for 8429 is as follows:
    84.29 Self-propelled bulldozers, angledozers, graders, levellers, scrapers, mechanical shovels, excavators, shovel loaders, tamping machines and road rollers.

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DC_VAL using both SUPLY_CO_NAME and PRDT_DC_VAL.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Input Example:
    PRDT_DC_VAL: COSMELAN 2 (FACE CREAM)
    SUPLY_CO_NAME: MESOESTETIC, S.L.
    Output Example: 'This product is called COSMELAN 2, a face cream designed to reduce pigmentation and improve skin tone.
    It is often used for treating melasma and other skin discolorations by inhibiting melanin production.'


    PRODUCT DESCRIPTION(PRDT_DC_VAL) : {prdt_desc}
    SUPPLEYER NAME(SUPLY_CO_NAME) : {suply_name}
    """
)

generate_chain_bldata = generate_prompt_bldata | chat_openai

In [ ]:
bldata_preprocessed_dict_openai={}

for index, row in tqdm(bl_data_sampled.iterrows()):
  response_bldata = generate_chain_bldata.invoke({"prdt_desc": row['PRDT_DC_VAL'], "suply_name":row['SUPLY_CO_NAME']})
  if index%100==0:
    print(f"\nProduct Name : {row['PRDT_DC_VAL']} \n")
    print(response_bldata.content, '\n')
  bldata_preprocessed_dict_openai[row['PRDT_DC_VAL']]=[response_bldata.content]

1it [00:01,  1.06s/it]


Product Name : 2700 BOXES FROZEN STRAWBERRY (FRESA CONGELADA) 0811.10.90 .00 PO: 31508-7 TERMOREGISTR OS:75297640-75297642<br/> 

'This product is described as 2700 boxes of frozen strawberries (fresa congelada). Frozen strawberries are typically used in various culinary applications, including desserts, smoothies, and as toppings for cereals or yogurt. They are harvested at peak ripeness and then quickly frozen to preserve their flavor and nutritional value. This product is supplied by MEBOL GF S.A.C., a company that specializes in the distribution of frozen fruits and vegetables.' 



101it [02:25,  1.18s/it]


Product Name : CHRISTMAS GIFT BAG THIS SHIPMENT DOES NOT C ONTAIN ANY SOLID WOOD PACKING MATERIAL SHIPP ER S LOAD COUNT & SEALED<br/> 

This product is a Christmas gift bag, which is typically used for packaging gifts during the holiday season. It is designed to be festive and appealing, making it a popular choice for presenting gifts to friends and family. The note about the shipment not containing any solid wood packing material indicates compliance with international shipping regulations, ensuring that the packaging is environmentally friendly and safe for transport. The supplier, XIEXIN TRADING CO., LTD, specializes in trading various products, including seasonal items like gift bags. 



201it [04:47,  1.47s/it]

...

401it [09:46,  2.58s/it]


Product Name : HEADPHONES 8518302000 VACUUM CUP 7323930080 THE SAME SHIPPER S DECLARATION THIS SHIPMENT DOES NOT CONTAIN ANY SOLID WOOD PACKING MATERIALS<br/> 

The provided input seems to contain multiple product descriptions and a supplier name that doesn't directly relate to the products listed. However, I can extract and explain the relevant parts.

1. **PRDT_DC_VAL**: HEADPHONES 8518302000
   - This product refers to headphones, which are audio devices worn on or over the ears to listen to sound. They are commonly used with various electronic devices such as smartphones, computers, and music players for personal audio enjoyment.

2. **PRDT_DC_VAL**: VACUUM CUP 7323930080
   - A vacuum cup is a device used to create a vacuum seal, often utilized in various applications including industrial processes, packaging, and food storage. It helps in preserving freshness by removing air from the container.

3. **SUPLY_CO_NAME**: 4500
   - The supplier name appears to be a numerical identifi

501it [12:16,  1.20s/it]


Product Name : PHONE CASE 3926909090 THIS SHIPMENT DOES NOT CONTAIN ANY SOLID WOOD PACKING MATERIALS<br/> 

This product is a phone case, categorized under HS Code 3926, which pertains to articles made of plastics and articles of other materials. The specific code 9090 indicates a particular type of plastic product. The supplier for this phone case is 45G1, which specializes in providing various plastic goods. This shipment is noted to not contain any solid wood packing materials, ensuring compliance with international shipping regulations regarding pest control and environmental safety. 



546it [15:13,  1.86s/it]

In [ ]:
joblib.dump(bldata_preprocessed_dict_openai, 'bldata_preprocessed_dict_openai.pkl')

In [ ]:
bldata_preprocessed_dict_openai_ext=joblib.load('bldata_preprocessed_dict_openai.pkl')

In [ ]:
parser.get_format_instructions()

In [ ]:
extract_prompt_bldata = PromptTemplate.from_template(
    """Extract product type and keywords, phrases from input, based on following rules:
    - Strictly exclude any information about capacity or volume
    - Generate a list of keywords related to product's main feature, but strictly limit the number to {{keyword_num}} or less. Do not exceed this limit under any circumstances.
    - Generate a list of phrases related to product's main feature.

    INPUT : {input}
    FORMAT :{{ "type": "<product type>", "keyword": [<keyword1>. <keyword2>, ...], "phrase" : [<phrase1>. <phrase2>, ...]}}
    """
)

extract_prompt_bldata= extract_prompt_bldata.partial(keyword_num=keyword_num)

extract_chain_bldata = extract_prompt_bldata | chat_openai | parser

In [ ]:
for i, (key, value) in tqdm(enumerate(bldata_preprocessed_dict_openai_ext.items())):
  response_bldata = extract_chain_bldata.invoke({"input": value[0]})
  if i%100==0:
    print(response_bldata)
  bldata_preprocessed_dict_openai_ext[key].append(response_bldata.type)
  bldata_preprocessed_dict_openai_ext[key].append(response_bldata.keyword)
  bldata_preprocessed_dict_openai_ext[key].append(response_bldata.phrase)

In [ ]:
joblib.dump(bldata_preprocessed_dict_openai_ext, 'bldata_preprocessed_dict_openai_ext.pkl')

In [ ]:
bldata_preprocessed_openai=pd.DataFrame.from_dict(bldata_preprocessed_dict_openai_ext, orient='index', columns=['product_description', 'type', 'keyword', 'phrase'])
bldata_preprocessed_openai.reset_index(inplace=True)
bldata_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
bldata_preprocessed_openai.to_csv('bldata_preprocessed_openai.csv', index=False)

display(bldata_preprocessed_openai.head())

# 수출한 기업은 다르지만, 수출한 품목의 이름은 같은 경우가 있어 불일치 발생
(중복 제거됨)

In [ ]:
# 같은 품목 중 다른 기업이 수출한 이력이 있는 경우, 다른 수출건으로 집계됨

bl_data_sampled.shape

In [ ]:
# 제품 이름에 기반해, 제품 수입 확률이 높은 해외 바이어 매칭이므로
# 중복되는 제품 이름은 하나의 키워드 및 임베딩으로 통일

len(bldata_preprocessed_dict_openai)

# 임베딩

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 오픈소스 임베딩 모델 다운로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [ ]:
def text_embedding(text):
  embedding_text=embeddings.embed_query(text)
  return np.array(embedding_text).reshape(1,-1)

In [ ]:
bldata_preprocessed_openai['product_description'][0]

#  1. 전체 문장 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['description_summary']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['product_description']

# 전체 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_description', 'bldata_product_name', 'bldata_description', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_description': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_description']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())

In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v1.csv', index=False)

# 2. 제품 유형 유사도 필터 후 전체 문장 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

# 제품 유형을 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 전체 문장을 임베딩
buykorea_preprocessed_openai['buykorea_description_embedding']=buykorea_preprocessed_openai['description_summary'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_description_embedding']=bldata_preprocessed_openai['product_description'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_description', 'bldata_product_name',
                                           'bldata_type', 'bldata_description', 'type_cosine_similarity', 'description_cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_description': [row['description_summary']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'description_cosine_similarity' : [0]*bldata_sample_len})

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_description_embedding=row['buykorea_description_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_type_embedding=row_bl['bldata_type_embedding']
    bldata_description_embedding=row_bl['bldata_description_embedding']

    type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
    description_similarity=cosine_similarity(buykorea_description_embedding, bldata_description_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_type']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
    similarity_df.loc[row_count, 'description_cosine_similarity']=description_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity', 'description_cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v2.csv', index=False)

# 3. 제품 유형 유사도와 키워드 5개 유사도 순 나열

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

# 제품 유형을 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['keyword_sentence']=buykorea_preprocessed_openai['keyword'].progress_apply(lambda x :x.strip('[]'))
bldata_preprocessed_openai['keyword_sentence']=bldata_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))

buykorea_preprocessed_openai['buykorea_keyword_embedding']=buykorea_preprocessed_openai['keyword_sentence'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_keyword_embedding']=bldata_preprocessed_openai['keyword_sentence'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_keyword', 'bldata_product_name', 'bldata_type', 'bldata_keyword', 'type_cosine_similarity' ,'keyword_cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_keyword': [row['keyword_sentence']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'keyword_cosine_similarity' : [0]*bldata_sample_len})

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_keyword_embedding=row['buykorea_keyword_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_type_embedding=row_bl['bldata_type_embedding']
    bldata_keyword_embedding=row_bl['bldata_keyword_embedding']

    type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
    keyword_similarity=cosine_similarity(buykorea_keyword_embedding, bldata_keyword_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['keyword_sentence']
    similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
    similarity_df.loc[row_count, 'keyword_cosine_similarity']=keyword_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity', 'keyword_cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v3.csv', index=False)

# 4. 제품 유형 유사도와 전체 문장 + 핵심 구절 n개 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['phrase_sentence']=buykorea_preprocessed_openai['phrase'].progress_apply(lambda x : x.strip('[]'))
bldata_preprocessed_openai['phrase_sentence']=bldata_preprocessed_openai['phrase'].progress_apply(lambda x : x.strip('[]'))

# 제품 유형 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 전체 문장 + 구절로 구성된 문장을 임베딩

buykorea_preprocessed_openai['keyword_sentence']=buykorea_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))
bldata_preprocessed_openai['keyword_sentence']=bldata_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))

buykorea_preprocessed_openai['buykorea_description_phrase_embedding']=buykorea_preprocessed_openai['description_summary']+buykorea_preprocessed_openai['phrase_sentence'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_description_phrase_embedding']=bldata_preprocessed_openai['product_description']+bldata_preprocessed_openai['phrase_sentence'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
import re

openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_phrase_description',
                                           'bldata_product_name', 'bldata_type','bldata_phrase_description',
                                           'type_cosine_similarity', 'phrase_description_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_phrase_description': [row['description_summary']+row['phrase_sentence']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'phrase_description_cosine_similarity' : [0]*bldata_sample_len,
                              })

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_phrase_description_embedding=row['buykorea_description_phrase_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):
      bldata_type_embedding=row_bl['bldata_type_embedding']
      bldata_phrase_description_embedding=row_bl['bldata_description_phrase_embedding']

      type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
      phrase_description_similarity=cosine_similarity(buykorea_phrase_description_embedding, bldata_phrase_description_embedding)
      similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
      similarity_df.loc[row_count, 'bldata_type']=row_bl['type']
      similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
      similarity_df.loc[row_count, 'phrase_description_cosine_similarity']=phrase_description_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())



In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'],
                                                          group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity','phrase_description_cosine_similarity'], ascending=False))

openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.grozupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v4.csv', index=False)

# 5. 제품 유형 유사도와 핵심 구절 n개 임베딩 + 전체 문장 (순서 변화)

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['description_summary']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['product_description']

# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)


bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_description', 'bldata_product_name', 'bldata_description', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_description': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_description']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v6.csv', index=False)